## Crypto Price Analysis Using SQLite and Python
### Business Questions
This project aims to answer key questions relevant to cryptocurrency price analysis:

- What is the average price for each coin over the last 5 days?

- What is the highest price recorded for each coin?

- Which coin had the largest price increase over the period?

- What is the price volatility (standard deviation) for each coin?

- Which coin had the lowest average price?

These questions help investors and analysts evaluate performance, risk, and potential value across different cryptocurrencies.

### Step 1: Import Required Libraries

We begin by importing essential libraries:

sqlite3 for database operations

pandas for data manipulation and analysis

datetime for generating date ranges

In [126]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta


### Step 2: Create SQLite Database and Tables

This cell creates two tables:

coins: stores coin metadata

prices: stores daily price records linked to coins via foreign keys

In [127]:
db_path = "crypto_data.db"

with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS coins (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            symbol TEXT NOT NULL UNIQUE
        )
    """)
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS prices (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            coin_id INTEGER,
            date TEXT,
            price REAL,
            FOREIGN KEY (coin_id) REFERENCES coins(id)
        )
    """)


### Step 3: Insert Coins into the Database

In [128]:

coin_metadata = [
    (1, "Bitcoin", "BTC"),
    (2, "Ethereum", "ETH"),
    (3, "Solana", "SOL")
]

with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    cursor.executemany("INSERT OR IGNORE INTO coins (id, name, symbol) VALUES (?, ?, ?)", coin_metadata)
coin_bases = {
    "BTC": (1, 65000),
    "ETH": (2, 3500),
    "SOL": (3, 150)
}


### Step 4: Insert Daily Price Records

This cell generates 5 mock price records per coin using a simple formula to simulate realistic variation.

In [129]:
start_date = datetime(2025, 8, 15)
price_data = []

for symbol, (coin_id, base_price) in coin_bases.items():
    for i in range(5):
        date = (start_date + timedelta(days=i)).strftime("%Y-%m-%d")
        fluctuation = base_price * (1 + ((i - 2) * 0.01))  # ±2% centered on day 2
        price = round(fluctuation, 2)
        price_data.append((coin_id, date, price))

with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    cursor.executemany("INSERT INTO prices (coin_id, date, price) VALUES (?, ?, ?)", price_data)


with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM prices")
    rows = cursor.fetchall()
    for row in rows:
        print(row)


(1, 1, '2025-08-15', 63700.0)
(2, 1, '2025-08-16', 64350.0)
(3, 1, '2025-08-17', 65000.0)
(4, 1, '2025-08-18', 65650.0)
(5, 1, '2025-08-19', 66300.0)
(6, 2, '2025-08-15', 3430.0)
(7, 2, '2025-08-16', 3465.0)
(8, 2, '2025-08-17', 3500.0)
(9, 2, '2025-08-18', 3535.0)
(10, 2, '2025-08-19', 3570.0)
(11, 3, '2025-08-15', 147.0)
(12, 3, '2025-08-16', 148.5)
(13, 3, '2025-08-17', 150.0)
(14, 3, '2025-08-18', 151.5)
(15, 3, '2025-08-19', 153.0)
(16, 1, '2025-08-15', 63700.0)
(17, 1, '2025-08-16', 64350.0)
(18, 1, '2025-08-17', 65000.0)
(19, 1, '2025-08-18', 65650.0)
(20, 1, '2025-08-19', 66300.0)
(21, 2, '2025-08-15', 3430.0)
(22, 2, '2025-08-16', 3465.0)
(23, 2, '2025-08-17', 3500.0)
(24, 2, '2025-08-18', 3535.0)
(25, 2, '2025-08-19', 3570.0)
(26, 3, '2025-08-15', 147.0)
(27, 3, '2025-08-16', 148.5)
(28, 3, '2025-08-17', 150.0)
(29, 3, '2025-08-18', 151.5)
(30, 3, '2025-08-19', 153.0)
(31, 1, '2025-08-15', 63700.0)
(32, 1, '2025-08-16', 64350.0)
(33, 1, '2025-08-17', 65000.0)
(34, 1, '2025-08

### Step 5: Average Price per Coin
This query calculates the average price for each coin over the 5-day period.

In [130]:
with sqlite3.connect(db_path) as conn:
    avg_df = pd.read_sql("""
        SELECT c.name, c.symbol, ROUND(AVG(p.price), 2) AS avg_price
        FROM prices p
        JOIN coins c ON p.coin_id = c.id
        GROUP BY p.coin_id
    """, conn)

avg_df


,name,symbol,avg_price
0,Bitcoin,BTC,65000.0
1,Ethereum,ETH,3500.0
2,Solana,SOL,150.0


### Step 6: Highest Price per Coin

This query identifies the highest price recorded for each coin.

In [131]:
with sqlite3.connect(db_path) as conn:
    max_df = pd.read_sql("""
        SELECT c.name, c.symbol, MAX(p.price) AS max_price
        FROM prices p
        JOIN coins c ON p.coin_id = c.id
        GROUP BY p.coin_id
    """, conn)

max_df


,name,symbol,max_price
0,Bitcoin,BTC,66300.0
1,Ethereum,ETH,3570.0
2,Solana,SOL,153.0


### Step 7: Largest Price Increase

This query calculates the price change over the 5-day period to identify momentum

In [132]:
with sqlite3.connect(db_path) as conn:
    change_df = pd.read_sql("""
        SELECT c.symbol,
               MAX(p.price) - MIN(p.price) AS price_change
        FROM prices p
        JOIN coins c ON p.coin_id = c.id
        GROUP BY p.coin_id
        ORDER BY price_change DESC
    """, conn)

change_df


,symbol,price_change
0,BTC,2600.0
1,ETH,140.0
2,SOL,6.0


### Step 8: Price Volatility (Standard Deviation)

In [133]:
with sqlite3.connect(db_path) as conn:
    df = pd.read_sql("""
        SELECT c.symbol, p.price
        FROM prices p
        JOIN coins c ON p.coin_id = c.id
    """, conn)

volatility = df.groupby("symbol")["price"].std().round(2)
volatility


symbol
BTC    951.50
ETH     51.23
SOL      2.20
Name: price, dtype: float64

### Step 9: Coin with Lowest Average Price

In [134]:
with sqlite3.connect(db_path) as conn:
    low_avg_df = pd.read_sql("""
        SELECT c.symbol, ROUND(AVG(p.price), 2) AS avg_price
        FROM prices p
        JOIN coins c ON p.coin_id = c.id
        GROUP BY p.coin_id
        ORDER BY avg_price ASC
        LIMIT 1
    """, conn)

low_avg_df


,symbol,avg_price
0,SOL,150.0


### Step 10: View All Price Records

In [135]:
with sqlite3.connect(db_path) as conn:
    full_df = pd.read_sql("""
        SELECT c.name, c.symbol, p.date, p.price
        FROM prices p
        JOIN coins c ON p.coin_id = c.id
        ORDER BY p.date
    """, conn)

full_df


,name,symbol,date,price
0,Bitcoin,BTC,2025-08-15,63700.0
1,Ethereum,ETH,2025-08-15,3430.0
2,Solana,SOL,2025-08-15,147.0
3,Bitcoin,BTC,2025-08-15,63700.0
4,Ethereum,ETH,2025-08-15,3430.0
5,Solana,SOL,2025-08-15,147.0
6,Bitcoin,BTC,2025-08-15,63700.0
7,Ethereum,ETH,2025-08-15,3430.0
8,Solana,SOL,2025-08-15,147.0
9,Bitcoin,BTC,2025-08-16,64350.0
